# Creating Synthetic Data



1. Load csv files using dspy data loader
2. Create signature for DSPy module
3. Create module for generation
4. Use a loop to iterate through all file, option combinations

In [1]:
pip install dspy-ai

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\dbake\Documents\GREP_Boi\CS541_CS441_AI_Final_Project\SyntheticData\synth-env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import dspy
from dspy.datasets import DataLoader

dl = DataLoader()

grep_options_dataset = dl.from_csv("./data/grep_options.csv")
sample_files_dataset = dl.from_csv("./data/sample_files3.csv")

In [3]:
print(grep_options_dataset[1])
print(sample_files_dataset[1])

Example({'Options': '-a', 'Description': 'Treats binary files as text'}) (input_keys=set())
Example({'File_Path': '/var/log/auth.lo?', 'Description': 'Authentication log file. It is useful for recording system authentication attempts, identifying intrusion attempts and capturing relevant log events.', 'File': '/var/log/auth.log\n\nJan  1 00:00:01 localhost sshd[1234]: Accepted password for user123 from 192.168.1.1 port 12345 ssh2'}) (input_keys=set())


In [4]:
# define typed signature
import os

from dotenv import load_dotenv

load_dotenv("../.env")

GOOGLE_API_KEY = os.getenv("PAID_GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

gemini_flash = dspy.Google(model="gemini-1.5-flash-latest", temperature=0)
gpt35_instruct = dspy.OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.6)
gpt35_turbo = dspy.OpenAI(model="gpt-3.5-turbo",temperature=0)

dspy.settings.configure(lm=gemini_flash)

### Create generation module

In [5]:
from pydantic import BaseModel, Field, conlist
from typing import List, NamedTuple

class GrepPair(NamedTuple):
    command: str
    description: str

class Output(BaseModel):
    commands: conlist(GrepPair, min_length=10, max_length=10) = Field(..., description="List of 10 pairs of grep commands and a natural language description ")
    
class QuestionOutput(BaseModel):
    summaries: conlist(str, min_length=10, max_length=10) = Field(..., description="List of 10 natural language descriptions of something to look for with grep")

class GrepOutput(BaseModel):
    commands: conlist(str, min_length=11, max_length=10) = Field(..., description="List of 10 grep commands that would be useful")

class CommentInput(BaseModel):
    commands: conlist(str, min_length=10, max_length=10) = Field(..., description="List of 10 grep commands")

class CommentOutput(BaseModel):
    descriptions: conlist(str, min_length=10, max_length=10) = Field(..., description="corresponding list of natural language descriptions for the grep commands")

class test_generation(dspy.Signature):
    """Given the contents of a file, and the file name as well as a grep option as input create a list of useful and interesting grep commands"""
    file = dspy.InputField(desc="The file to make grep commands for")
    filename = dspy.InputField(desc="The filename")
    option = dspy.InputField(desc="The grep option to use")
    option_description = dspy.InputField(desc="A description of the grep option")
    commands: Output = dspy.OutputField()

class test_generation_simple(dspy.Signature):
    """Given the contents of a file, and the file name as well as a grep option as input, create a useful and interesting grep command"""
    file = dspy.InputField(desc="The file to make grep commands for")
    filename = dspy.InputField(desc="The filename")
    option = dspy.InputField(desc="The grep option to use")
    option_description = dspy.InputField(desc="A description of the grep option")
    commands: GrepOutput = dspy.OutputField()

generate_answer = dspy.TypedPredictor(test_generation)
generate_simple_answer = dspy.Predict(test_generation_simple)

Simple test

In [6]:
grep_option = grep_options_dataset[0]
file = sample_files_dataset[0]
fileFile = "```"+file.File+"```"
print(fileFile)

```/var/log/syslog```


In [9]:
with dspy.context(lm=gemini_flash):
    resp = generate_answer(file=file.File, filename=file.File_Path, option=grep_option.Options, option_description=grep_option.Description)

the_command=resp.commands

print(resp.commands)


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [155]:
# for idx, grep_pair in enumerate(resp.commands):
#     print(grep_pair)
#     print(grep_pair[1][idx].command)
#     print(grep_pair[1][idx].description)
#     # print(grep_pair.command, grep_pair.description)

for intermediate_grep in resp.commands:
            grep_pairs = intermediate_grep[1]
            for grep_pair in grep_pairs:
                command = grep_pair.command
                description = grep_pair.description
                print(command)

print(resp.commands)

grep -A 5 'kernel: [A-Z]+' /var/log/syslog
grep -A 10 'user.notice' /var/log/syslog
grep -A 2 'sshd: Authentication failure' /var/log/syslog
grep -A 3 'crontab: Reloading' /var/log/syslog
grep -A 1 'audit: type=SYSCALL' /var/log/syslog
grep -A 5 'pam_unix(sshd:auth): authentication failure' /var/log/syslog
grep -A 2 'systemd-timesyncd: Time synchronized' /var/log/syslog
grep -A 1 'NetworkManager: Activation' /var/log/syslog
grep -A 3 'apparmor: [A-Z]+' /var/log/syslog
grep -A 5 'SELinux: [A-Z]+' /var/log/syslog
commands=[GrepPair(command="grep -A 5 'kernel: [A-Z]+' /var/log/syslog", description='Show 5 lines of context after lines containing kernel messages with uppercase letters.'), GrepPair(command="grep -A 10 'user.notice' /var/log/syslog", description='Show 10 lines of context after lines containing user.notice messages.'), GrepPair(command="grep -A 2 'sshd: Authentication failure' /var/log/syslog", description='Show 2 lines of context after lines containing sshd authentication fai

## Define generation loop
Its just a fully connected pair-wise connection of options to example files

In [119]:
def save_results_to_csv(filename, results, mode='w'):
    fieldnames = ['command', 'description', 'file_path', 'option']
    with open(filename, mode, newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if mode == 'w':
            writer.writeheader()
        
        for result in results:
            resp, file_path, option = result
            for intermediate_grep in resp.commands:
                grep_pairs = intermediate_grep[1]
                for grep_pair in grep_pairs:
                    writer.writerow({
                        'command': grep_pair.command,
                        'description': grep_pair.description,
                        'file_path': file_path,
                        'option': option
                    })

In [80]:
# check if file already exists
# 

save_results_to_csv(filename, [], mode='w')
for option_idx, grep_option in enumerate(grep_options_dataset):
    for file_idx, sample_file in enumerate(sample_files_dataset):
        # Put code here

Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A', 'Description': 'Prints NUM lines of trailing context after matching lines'}) (input_keys=set())
Example({'Options': '-A'

In [111]:
pip install ratelimit

You should consider upgrading via the 'C:\Users\dbake\Documents\GREP_Boi\CS541_CS441_AI_Final_Project\SyntheticData\synth-env\Scripts\python.exe -m pip install --upgrade pip' command.


Using legacy 'setup.py install' for ratelimit, since package 'wheel' is not installed.

    Running setup.py install for ratelimit: started
    Running setup.py install for ratelimit: finished with status 'done'


### Complex multithreaded approach

In [143]:
import os
import csv
import time
import signal
from concurrent.futures import ThreadPoolExecutor, as_completed, TimeoutError
from ratelimit import limits, sleep_and_retry
import random  # For adding jitter to the retry sleep time

# Constants
REQUESTS_PER_MINUTE = 500
SECONDS_PER_MINUTE = 60
MAX_WORKERS = 8  # Adjust based on performance
MAX_RETRIES = 3  # Maximum number of retry attempts
TIMEOUT_SECONDS = 60  # Maximum time allowed for each call

# Rate limiting decorator
@sleep_and_retry
@limits(calls=REQUESTS_PER_MINUTE, period=SECONDS_PER_MINUTE)
def rate_limited_function(option_idx, grep_option, file_idx, sample_file):
    with dspy.context(lm=gemini_flash):
        smfp = sample_file.File_Path
        smf = sample_file.File
        grpo = grep_option.Options
        grpod = grep_option.Description
        resp = generate_answer(file=smf, filename=smfp, option=grpo, option_description=grpod)
    return (resp, smfp, grpo)

def rate_limited_function_with_retries(option_idx, grep_option, file_idx, sample_file, max_retries=MAX_RETRIES):
    attempts = 0
    while attempts < max_retries:
        try:
            start_time = time.time()
            result = rate_limited_function(option_idx, grep_option, file_idx, sample_file)
            end_time = time.time()
            print(f"Processed option {option_idx}, file {file_idx} in {end_time - start_time:.2f} seconds")
            return result
        except Exception as e:
            attempts += 1
            if attempts >= max_retries:
                print(f"Failed to process (option_idx={option_idx}, file_idx={file_idx}) after {max_retries} attempts: {e}")
                return None  # or some other placeholder indicating failure
            backoff_time = min(2 ** attempts, 60)  # Exponential backoff with a max of 60 seconds
            print(f"Retrying (option_idx={option_idx}, file_idx={file_idx}), attempt {attempts}")
            time.sleep(backoff_time + random.uniform(0, 1))  # Adding jitter

def save_result_to_csv(filename, result, mode='a'):
    fieldnames = ['command', 'description', 'file_path', 'option']
    with open(filename, mode, newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if mode == 'w':
            writer.writeheader()

        resp, file_path, option = result
        for intermediate_grep in resp.commands:
            grep_pairs = intermediate_grep[1]
            for grep_pair in grep_pairs:
                writer.writerow({
                    'command': grep_pair.command,
                    'description': grep_pair.description,
                    'file_path': file_path,
                    'option': option
                })

def main(grep_options_dataset, sample_files_dataset):
    filename = 'synthetic_data3.csv'

    # Initialize the CSV file with headers if it doesn't already exist
    if not os.path.exists(filename):
        save_result_to_csv(filename, [], mode='w')

    for option_idx, grep_option in enumerate(grep_options_dataset):
        for file_idx, sample_file in enumerate(sample_files_dataset):
            print(f"Processing option {option_idx}, file {file_idx}")  # Print progress
            try:
                with ThreadPoolExecutor(max_workers=1) as executor:
                    future = executor.submit(rate_limited_function_with_retries, option_idx, grep_option, file_idx, sample_file)
                    result = future.result(timeout=TIMEOUT_SECONDS)
                    if result:
                        save_result_to_csv(filename, result, mode='a')
                        print(f"Saved result for option {option_idx}, file {file_idx}")  # Print confirmation
            except TimeoutError:
                print(f"Timeout processing option {option_idx}, file {file_idx}")
            except Exception as e:
                print(f"An error occurred while processing option {option_idx}, file {file_idx}: {e}")



### Simpler 1 request at a time

In [160]:
import os
import csv
import time
from ratelimit import limits, sleep_and_retry
import random  # For adding jitter to the retry sleep time

# Constants
REQUESTS_PER_MINUTE = 500
SECONDS_PER_MINUTE = 60
MAX_RETRIES = 3  # Maximum number of retry attempts
TIMEOUT_SECONDS = 60  # Maximum time allowed for each call

# Rate limiting decorator
@sleep_and_retry
@limits(calls=REQUESTS_PER_MINUTE, period=SECONDS_PER_MINUTE)
def rate_limited_function(option_idx, grep_option, file_idx, sample_file):
    with dspy.context(lm=gemini_flash):
        smfp = sample_file.File_Path
        smf = sample_file.File
        grpo = grep_option.Options
        grpod = grep_option.Description
        resp = generate_answer(file=smf, filename=smfp, option=grpo, option_description=grpod)
    return (resp, smfp, grpo)

def rate_limited_function_with_retries(option_idx, grep_option, file_idx, sample_file, max_retries=MAX_RETRIES):
    attempts = 0
    while attempts < max_retries:
        try:
            start_time = time.time()
            result = rate_limited_function(option_idx, grep_option, file_idx, sample_file)
            end_time = time.time()
            # print(f"Processed option {option_idx}, file {file_idx} in {end_time - start_time:.2f} seconds")
            return result
        except Exception as e:
            attempts += 1
            if attempts >= max_retries:
                print(f"Failed to process (option_idx={option_idx}, file_idx={file_idx}) after {max_retries} attempts: {e}")
                return None  # or some other placeholder indicating failure
            backoff_time = min(2 ** attempts, 60)  # Exponential backoff with a max of 60 seconds
            print(f"Retrying (option_idx={option_idx}, file_idx={file_idx}), attempt {attempts}")
            time.sleep(backoff_time + random.uniform(0, 1))  # Adding jitter

def save_result_to_csv(filename, result, mode='a'):
    fieldnames = ['command', 'description', 'file_path', 'option']
    with open(filename, mode, newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if mode == 'w':
            writer.writeheader()

        resp, file_path, option = result
        for intermediate_grep in resp.commands:
            grep_pairs = intermediate_grep[1]
            for grep_pair in grep_pairs:
                writer.writerow({
                    'command': grep_pair.command,
                    'description': grep_pair.description,
                    'file_path': file_path,
                    'option': option
                })

def main(grep_options_dataset, sample_files_dataset):
    filename = './data/synthetic_data.csv'

    # Initialize the CSV file with headers if it doesn't already exist
    if not os.path.exists(filename):
        save_result_to_csv(filename, [], mode='w')

    for option_idx, grep_option in enumerate(grep_options_dataset):
        for file_idx, sample_file in enumerate(sample_files_dataset):
            # print(f"Processing option {option_idx}, file {file_idx}")  # Print progress
            try:
                result = rate_limited_function_with_retries(option_idx, grep_option, file_idx, sample_file)
                if result:
                    save_result_to_csv(filename, result, mode='a')
                    # print(f"Saved result for option {option_idx}, file {file_idx}")  # Print confirmation
            except Exception as e:
                print(f"An error occurred while processing option {option_idx}, file {file_idx}: {e}")

In [ ]:
results = main(grep_options_dataset, sample_files_dataset)

Retrying (option_idx=5, file_idx=1), attempt 1
Retrying (option_idx=5, file_idx=1), attempt 2
Failed to process (option_idx=5, file_idx=1) after 3 attempts: ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'commands': 'JSONDecodeError("Unrecognized escape sequence when decoding \'string\'")'})
Retrying (option_idx=5, file_idx=51), attempt 1
Retrying (option_idx=5, file_idx=51), attempt 2


ERROR:backoff:Giving up request(...) after 1 tries (google.api_core.exceptions.ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).)


Retrying (option_idx=5, file_idx=83), attempt 1


ERROR:backoff:Giving up request(...) after 1 tries (google.api_core.exceptions.ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).)


Retrying (option_idx=6, file_idx=125), attempt 1
Retrying (option_idx=8, file_idx=15), attempt 1
Retrying (option_idx=8, file_idx=29), attempt 1


...

Failed to process (option_idx=14, file_idx=23) after 3 attempts: 429 Resource has been exhausted (e.g. check quota).


ERROR:backoff:Giving up request(...) after 1 tries (google.api_core.exceptions.ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).)


Retrying (option_idx=14, file_idx=24), attempt 1


ERROR:backoff:Giving up request(...) after 1 tries (google.api_core.exceptions.ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).)


Retrying (option_idx=14, file_idx=24), attempt 2


In [72]:
with dspy.context(lm=gpt35_instruct):
    resp = generate_simple_answer(file=file.File, filename=file.File_Path, option="-Eln", option_description=grep_option.Description)

the_command=resp.commands

print(resp.commands)

grep -Eln "error" /var/log/syslo?.log


In [54]:
print(resp)

Prediction(
    command='---\n\nFile: /var/log/syslog\n\nFilename: /var/log/syslo?.log\n\nOption: -A\n\nOption Description: Prints NUM lines of trailing context after matching lines\n\nCommand: `grep -A 5 "kernel: [A-Z]" /var/log/syslo?.log`'
)
